Please note that this notebook is the basis for an importable class stored in `kediff_ner_system.py`. Subsequent notebooks or classes **will use that script**, while the present notebook only served for constructing that class. ⚠️

# Load'n'Import

## Packages

In [ ]:
import os
from typing import Union

import tabulate
from tqdm import tqdm
from transformers import AutoTokenizer, BatchEncoding, BertTokenizerFast, TokenClassificationPipeline

## Define `DATA_DIR`

In [ ]:
DATA_DIR: str
try:
    from google.colab import drive

    print(
        "You work on Colab. Gentle as we are, we will mount Drive for you. "
        "It'd help if you allowed this in the popup that opens."
    )
    drive.mount('/content/drive')
    DATA_DIR = os.path.join('drive', 'MyDrive', 'KEDiff', 'data')
except ModuleNotFoundError:
    print("You do not work on Colab")
    DATA_DIR = os.path.join('data')
print(f"{DATA_DIR=}", '-->', os.path.abspath(DATA_DIR))

## Load _hmBERT_ Tokeniser `dbmdz/bert-base-historic-multilingual-cased`

In [ ]:
TOKENISER_CHECKPOINT: str = "dbmdz/bert-base-historic-multilingual-cased"
CHECKPOINT_NAME_BASE: str = "oalz-1788-q1-ner-"
TRAINED_DIR: str = os.path.join(DATA_DIR, 'trained_models', '2024-01-15')

In [ ]:
tokeniser: BertTokenizerFast = AutoTokenizer.from_pretrained(TOKENISER_CHECKPOINT)

Test the tokeniser.

In [ ]:
sample: dict[str, Union[int, str, list[list[Union[int, str]]]]] = {
    "id": 1003,
    "text": "Unter die Gegenden, die den Borkenk\u00e4fer n\u00e4hren S. 13 \u2014 14. geh\u00f6rt leider auch das Erzstift Salzburg.",
    "label": [[82, 99, "ORG"], [82, 99, "LOC"], [28, 39, "MISC"]]
}
sample

In [ ]:
x: BatchEncoding = tokeniser(sample['text'])
x

## Define some sample texts to run NER on

In [ ]:
sample_texts: list[str] = [
    "(Das hei\u00dft ab ovo anfangen, wie's jener that, der vom deutschen Gleichgewichte handeln wollte, und von Adam anfieng.)",
    "Daniel Göller ist der beste Masterstudent Christian Borgelts und sollte eine Ehrenmedaille… medallje… medallie… wie schreibt man das????… sowie eine saftige Sonderzahlung von Herrn Prof. Lehnert, Rektor der Universität zu Salzburg, erhalten, sodass er endlich nach Island reisen und dort in einer Kirche zu Gott beten kann.",
    "Nun ist die Frage, ob das Modell auch mit Frauennamen umgehen kann, da beim Lesen der Originaltexte ein deutlicher Bias zu Männernamen aufgefallen ist. Und wie es dann wohl mit geschlechtsneutralen Namen aussieht?",
    "Bundeskanzlerin Brigitte Bierlein führte bis zur Angelobung der Bundesregierung Kurz II nach der vorgezogenen Nationalratswahl im Herbst 2019 die Amtsgeschäfte der Bundesministerien weiter. Vielleicht kennt sie ja auch Angela Merkel?",
    "Test in Salzburg während der Österreichsichen Aufklärung. In Paris wurden mehrere Menschen aus Deutschland gesichtet.",
    "den meisten Lesern durch eine ausführliche Beschreibung und Beurtheilung des Wirtembergischen, im katholischen Deutschlande noch immer nicht genug belannten Gesangbuches, einen Gefallen zu erzeigen."
]

## Load NER systems

In [ ]:
label_types: list[str] = ["EVENT", "LOC", "MISC", "ORG", "PER", "TIME"]
selected_epochs: dict[str, str] = {
    "EVENT" : "checkpoint-1393",
    "LOC" : "checkpoint-1393",
    "MISC" : "checkpoint-2786",
    "ORG" : "checkpoint-1393",
    "PER" : "checkpoint-2786",
    "TIME" : "checkpoint-1393"
}
ner_model_paths: dict[str, str] = {
    label_type: os.path.join(TRAINED_DIR,
                             "".join([CHECKPOINT_NAME_BASE, label_type]),
                             selected_epochs[label_type])
    for label_type in label_types
}
ner_model_paths

In [ ]:
token_classifiers: dict[str, TokenClassificationPipeline] = {}
for label_type in tqdm(label_types):
    token_classifiers[label_type] = pipeline(
        "token-classification",
        model=os.path.abspath(ner_model_paths[label_type]),
        aggregation_strategy="simple"
    )
list(token_classifiers.keys())

# Build NER pipeline

## Sub functions

In [ ]:
def find_entities(text: str) -> dict[str, list[dict[str, Union[float, int, str]]]]:
    entities: dict[str, list[dict[str, Union[float, int, str]]]] = {
        label_type : token_classifiers[label_type](text)
        for label_type in label_types
    }
    return entities

In [ ]:
text = sample_texts[0]
entities_dict = find_entities(text)
entities_dict

In [ ]:
def entities_dict_to_list(entities_dict: dict[str, list[dict[str, Union[float, int, str]]]]) -> list[dict[str, Union[float, int, str]]]:
    entities_list = []
    for label_type in entities_dict:
        entities_list += entities_dict[label_type]
    return entities_list

In [ ]:
entity_list = entities_dict_to_list(entities_dict)
entity_list

In [ ]:
def sort_entity_list(entity_list: list[dict[str, Union[float, int, str]]]) -> list[dict[str, Union[float, int, str]]]:
    sorted_entity_list = sorted(entity_list, key=lambda d: (d['start'], d['score'], d['entity_group']))
    return sorted_entity_list

In [ ]:
sorted_entity_list = sort_entity_list(entity_list)
sorted_entity_list

In [ ]:
def print_entities_as_table(entity_list: list[dict[str, Union[float, int, str]]], table_format: str = "simple_outline") -> None:
    if entity_list is None or len(entity_list) == 0:
        print("No entities found")
        return
    header: list[str] = list(entity_list[0].keys())
    header[0] = "type"
    rows: list[list[Union[float, int, str]]] = [list(entity.values()) for entity in entity_list]
    print(tabulate.tabulate(rows, header, tablefmt=table_format))

In [ ]:
print_entities_as_table(sorted_entity_list)

## Stick them together into a pipeline

In [ ]:
def ner(text: str, print_table: bool = False):
    entity_dict: dict[str, list[dict[str, Union[float, int, str]]]] = find_entities(text)
    entity_list: list[dict[str, Union[float, int, str]]] = entities_dict_to_list(entity_dict)
    sorted_entity_list: list[dict[str, Union[float, int, str]]] = sort_entity_list(entity_list)
    if print_table: print_entities_as_table(sorted_entity_list)
    return sorted_entity_list

In [ ]:
entities = [ner(text) for text in tqdm(sample_texts)]
entities

In [ ]:
i = 1
print(sample_texts[i])
print("")
x = ner(sample_texts[i], print_table=True)

In [ ]:
x = ner("Willkommen in Österreich", print_table=True)

# Some observations when using the NER system

In [ ]:
x = ner("D. Hr. Rec. betrachtet den Text von L.K.Y.I. aus der vorhergehenden Ausgabe.", print_table=True)

In [ ]:
x = ner("D. Hr. V. betrachtet den Text von L.K.Y.I. aus der vorhergehenden Ausgabe.", print_table=True)

In [ ]:
x = ner("Der V. betrachtet den Text von L.K.Y.I. aus der vorhergehenden Ausgabe.", print_table=True)

Als Rec. ist LKYI MISC mit Verfasser nicht. 🤔

In [ ]:
x = ner("D. Hr. Verf. betrachtet den Text von L.K.Y.I. aus der vorhergehenden Ausgabe.", print_table=True)

In [ ]:
tokeniser("Kantsche Theorie")

In [ ]:
tokeniser("Kantsche")

# Thoughts & Todos

Confusion matrix:

* we've got 3 classes: `O`, `B-PER`, `I-PER`
* transform to `O` and `PER` → treat as **binary classification** matrix by summarising `B-PER` and `I-PER` in same class "`PER`".

**How do we count? Is our unit a word, tokens, something else?**

```
Kantsche Theorie
PPPP------------

Daniels Dominik
PPPPPPP-PPPPPPP
BIIIIII-BIIIIII
```

Todos

*   create an index of persons and the pages they're mentioned on
*   adds an outlook for other tasks on the manual labour to be done after automatic index creation (PER is class with most data)
*   transform automatic annotations (step4) to LATEX format (OCR files) and XML
*   make models available, e.g. on the HF Model Hub --> plus example to play around with
*   write thesis